In [1]:
import torch
from pprint import pprint
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
# Load the model and tokenizer
model_id = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [33]:
input_ids = tokenizer.encode("Hello, I'm a language model,", return_tensors="pt")
for i in range(4):
    # Get output probabilities
    with torch.no_grad():
        logits = model(input_ids).logits
    next_token_logits = logits[:, -1, :]
    probabilities = torch.nn.functional.softmax(next_token_logits, dim=-1)

    # Look at the top 5 candidates
    top_5 = torch.topk(probabilities, 5)
    top_5_token_ids = top_5.indices
    top_5_probabilities = top_5.values
    print("Candidates:")
    for token, prob in zip(top_5_token_ids[0], top_5_probabilities[0]):
        print(f"{tokenizer.decode(token.item())} with probability {prob.item()}")

    # Option 1: Choose the most likely candidate
    # next_token = top_5_token_ids[0, 0]
    # print(f"Choosing token {tokenizer.decode(next_token.item())}")

    # Option 2: Alternating between first and second candidate
    next_token = top_5_token_ids[0, i % 2]
    print(f"Choosing token {tokenizer.decode(next_token.item())}")

    # Add the new token to the input
    input_ids = torch.cat([input_ids, next_token[None, None]], dim=-1)

print(tokenizer.decode(input_ids[0], skip_special_tokens=True))

Candidates:
 and with probability 0.26030680537223816
 not with probability 0.13933220505714417
 here with probability 0.12296022474765778
 trained with probability 0.058082301169633865
 please with probability 0.03991934284567833
Choosing token  and
Candidates:
 I with probability 0.6940591335296631
 today with probability 0.053520094603300095
 here with probability 0.044369716197252274
 you with probability 0.02864724025130272
 this with probability 0.02864724025130272
Choosing token  today
Candidates:
 I with probability 0.6163644790649414
 we with probability 0.2569389343261719
, with probability 0.07361423969268799
's with probability 0.027081165462732315
 you with probability 0.008791966363787651
Choosing token  I
Candidates:
 will with probability 0.3834528923034668
'll with probability 0.2635430693626404
'm with probability 0.14106443524360657
'd with probability 0.05189470574259758
 would with probability 0.031475730240345
Choosing token 'll
Hello, I'm a language model, and to

## Text -> Binary

There are many options, this 5-bit version hacked together for demonstration by my new buddy Sonnet 3.5:

In [34]:
# Define the character set for 5-bit encoding
char_set = " abcdefghijklmnopqrstuvwxyz.,!?"
char_to_bin = {c: f"{i:05b}" for i, c in enumerate(char_set)}
bin_to_char = {v: k for k, v in char_to_bin.items()}

def encode_secret_message(txt):
    encoded = ""
    for c in txt:
        if c.lower() in char_set:
            if c != c.lower():
                encoded += "11111"
            encoded += char_to_bin[c.lower()]
      
    return encoded

def decode_secret_message(binary_string):
    chunks = [binary_string[i:i+5] for i in range(0, len(binary_string), 5)]
    decoded = ""
    uppercase_next = False
    for chunk in chunks:
        if chunk == "11111":
            uppercase_next = True
        else:
            char = bin_to_char.get(chunk, "")
            if uppercase_next:
                char = char.upper()
                uppercase_next = False
            decoded += char
    return decoded

t = "Hello World"
e = encode_secret_message(t)
d = decode_secret_message(e)
print(f"Original: {t}")
print(f"Encoded: {e}")
print(f"Decoded: {d}")

Original: Hello World
Encoded: 11111010000010101100011000111100000111111011101111100100110000100
Decoded: Hello World


## Encoding Binary with the choice of token:

In [28]:
def generate_steganographic_text(prompt, secret_message):
    # Encode start prompt and trim to multiple of 5
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    input_ids = input_ids[:, :((input_ids.size(1) // 5) * 5)]
    # Turn message into binary string and add tokens to input_ids
    encoded = encode_secret_message(secret_message)
    for bit in tqdm(encoded):
        with torch.no_grad():
            logits = model(input_ids).logits
        next_token_logits = logits[:, -1, :]
        probabilities = torch.nn.functional.softmax(next_token_logits, dim=-1)
        top_5_token_ids = torch.topk(probabilities, 5).indices
        next_token = top_5_token_ids[0, 0] if bit == "0" else top_5_token_ids[0, 1]
        input_ids = torch.cat([input_ids, next_token[None, None]], dim=-1)

    return tokenizer.decode(input_ids[0], skip_special_tokens=True)

# Example usage
prompt = "When I was young, we used to visit my grandmother in the countryside. " +\
         "She always bakes the most delicious pies."
secret_message = "Hawk at midnight!"
stego_text = generate_steganographic_text(prompt, secret_message)
print("Steganographic text:")
pprint(stego_text)

  0%|          | 0/90 [00:00<?, ?it/s]

Steganographic text:
('When I was young, we used to visit my grandmother in the countryside. She '
 'always bakes the most delicious pies. She used a special pie dish that I '
 'loved. It was a round, white ceramic pie plate. I loved the shape and the '
 'color. I loved the way it looked when she took the pie out of the oven. I '
 'loved how the pie crust looked on the plate.\n'
 'I have always wanted to have one. But I never did. I don’t know if I was '
 "afraid to ask for it, or I just didn't know how to get")


In [29]:
def decode_steganographic_text(text, start=0):
    ids = tokenizer.encode(text, return_tensors="pt")
    binary_string = ""
    with torch.no_grad():
            logits = model(ids).logits # << Note: just one forward pass to get all logits
    for i in range(start, len(ids[0])):
        next_token_logits = logits[:, i-1, :]
        top_5_token_ids = torch.topk(next_token_logits, 5).indices
        if ids[0][i] == top_5_token_ids[0, 0]:
            binary_string += "0"
        elif ids[0][i] == top_5_token_ids[0, 1]:
            binary_string += "1"
        else: 
            binary_string += "0" # Not part of top5 (happens in prompt) - add 0 to keep length right.
    return decode_secret_message(binary_string)

decode_steganographic_text(stego_text)

'ddh iHawk at midnigxt!'

In [30]:
decode_steganographic_text(stego_text, 25)

'Hawk at midnigxt!'

In [31]:
s = """In this video about steganography, we are talking to \
a person that is using steganography. We are asking him about \
the process and the reasons for doing it. We are asking about \
his experience and what are the benefits of using steganography.
In this video, I’m talking about the process and the reasons for \
doing it. We are talking to someone who is using stegnography and \
asking him about his experience. I’m asking him about the benefits \
and the process.\nStegano, or steganographic encryption is the practice \
where you embed..."""
decode_steganographic_text(s)

'd Thanks fer watching!'